In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

/home/lwd/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# One hot encoding for each char in 'hello'
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    #One cell RNN input_dimension(4) > output_dimension(2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype = np.float32)
    pp.pprint(x_data)
    outputs, __states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.27067557,  0.48000818]]], dtype=float32)


In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[-0.05660009,  0.37682691],
        [-0.07094067,  0.61848563],
        [-0.78341401, -0.44626895],
        [-0.64339137, -0.50236982],
        [ 0.67625624, -0.43305618]]], dtype=float32)


In [5]:
with tf.variable_scope('3_batches') as socpe:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.0588767 ,  0.06867331],
        [ 0.09078103,  0.07949297],
        [ 0.10506402,  0.08231065],
        [ 0.13180585,  0.08871793],
        [ 0.19988269,  0.04121359]],

       [[ 0.13245277,  0.01283496],
        [ 0.16326015, -0.01872113],
        [ 0.1504631 ,  0.00133999],
        [ 0.1559335 ,  0.01834938],
        [ 0.16036499,  0.03253094]],

       [[ 0.06553609,  0.02491119],
        [ 0.10711056,  0.04229171],
        [ 0.23815709,  0.04248779],
        [ 0.28479171,  0.0352258 ],
        [ 0.18080841,

In [6]:
with tf.variable_scope('3_batches_dynamic_lenght') as socpe:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length = [5,3,4] ,dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ -1.21367820e-01,   9.28923041e-02],
        [ -8.64079073e-02,  -2.21855342e-02],
        [ -6.43974394e-02,  -1.59500241e-01],
        [ -6.91191405e-02,  -2.61433780e-01],
        [ -1.49823681e-01,  -1.02285154e-01]],

       [[  1.28003294e-02,  -1.16549470e-01],
        [ -8.19841251e-02,   2.00632866e-03],
        [ -7.16223419e-02,  -1.37422889e-01],
        [  0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00]],

       [[ -1.98397487e-02,  -1.28085449e-01],
        [ -3.393086

In [7]:
with tf.variable_scope('initial_state') as socpe:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]], dtype=np.float32)   
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units = hidden_size, 
                             state_is_tuple = True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _stats = tf.nn.dynamic_rnn(
        cell, x_data, initial_state = initial_state ,dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ 0.09405788, -0.10112203],
        [ 0.17270254, -0.10455669],
        [ 0.12623718, -0.07821716],
        [ 0.10560169,  0.00030266],
        [-0.03832111, -0.07835191]],

       [[ 0.07387558, -0.05091054],
        [-0.0671995 , -0.14090177],
        [-0.0152092 , -0.05935089],
        [ 0.01457104,  0.03287275],
        [ 0.03793775,  0.11244818]],

       [[ 0.02705248,  0.08489441],
        [ 0.0484485 ,  0.15588853],
        [ 0.12158112,  0.03788715],
        [ 0.16549674, -0.02386612],
        [ 0.12482291,

In [8]:
#Create input data
batch_size = 3
squence_length = 5
input_dimension = 3
x_data = np.arange(45, dtype = np.float32).reshape(batch_size,
                                                   squence_length,input_dimension)
pp.pprint(x_data)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.],
        [ 12.,  13.,  14.]],

       [[ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.],
        [ 24.,  25.,  26.],
        [ 27.,  28.,  29.]],

       [[ 30.,  31.,  32.],
        [ 33.,  34.,  35.],
        [ 36.,  37.,  38.],
        [ 39.,  40.,  41.],
        [ 42.,  43.,  44.]]], dtype=float32)


In [9]:
with tf.variable_scope('generated_data') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 5, batch: 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state = initial_state, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -5.55621423e-02,   2.69159935e-02,  -1.06270470e-01,
          -1.41187683e-01,   1.72925457e-01],
        [  1.73611287e-02,   2.24070419e-02,  -3.58880877e-01,
          -4.73004609e-01,   4.54284966e-01],
        [  2.52195988e-02,   1.39435269e-02,  -4.36731637e-01,
          -6.53805017e-01,   6.02178752e-01],
        [  1.47487568e-02,   7.82546494e-03,  -4.17854905e-01,
          -7.74901927e-01,   6.52746558e-01],
        [  7.17662182e-03,   4.17426229e-03,  -3.86942953e-01,
          -8.57945859e-01,   6.82630479e-01]],

       [[  3.73089366e-04,   9.43667146e-06,  -2.92584628e-01,
          -7.30898261e-01,   7.14807510e-01],
        [  1.99320959e-04,   6.91834293e-06,  -3.31310809e-01,
          -8.81753385e-01,   7.33167470e-01],
        [  9.69751782e-05,   3.76223102e-06,  -3.06114733e-01,
          -9.31049764e-01,   7.44231045e-01],
        [  4.29016000e-05,   2.00915406e-06,  -2.80324340e-01,
          -9.51290488e-01,   7.51726449e-01],
        [  1.8016

In [10]:
# Create input data for Multi RNN
batch_size=3
sequence_length=5
input_dim=5

x_data = np.arange(75, dtype=np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)  

array([[[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [ 15.,  16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.,  24.]],

       [[ 25.,  26.,  27.,  28.,  29.],
        [ 30.,  31.,  32.,  33.,  34.],
        [ 35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.],
        [ 45.,  46.,  47.,  48.,  49.]],

       [[ 50.,  51.,  52.,  53.,  54.],
        [ 55.,  56.,  57.,  58.,  59.],
        [ 60.,  61.,  62.,  63.,  64.],
        [ 65.,  66.,  67.,  68.,  69.],
        [ 70.,  71.,  72.,  73.,  74.]]], dtype=float32)


In [11]:
with tf.variable_scope('MultilRNNcell') as scope:
    # make Rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] *3, state_is_tuple=True) #3 layers
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

dynamic rnn:  Tensor("MultilRNNcell/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[ 0.00116798,  0.00104186,  0.00352003, -0.00193464,  0.00021787],
        [ 0.00483343,  0.00111068,  0.01504887, -0.00629978,  0.00050146],
        [ 0.00902695,  0.00178937,  0.02721895, -0.01126098,  0.00169144],
        [ 0.01236872,  0.00376076,  0.03677143, -0.01611896,  0.00369036],
        [ 0.01452138,  0.00670948,  0.04323206, -0.02046258,  0.00610271]],

       [[ 0.0035787 , -0.00117903,  0.01193588, -0.00335925, -0.00073087],
        [ 0.00802279, -0.00129476,  0.02586171, -0.0080606 , -0.00040309],
        [ 0.01165426,  0.00054015,  0.03696573, -0.01310167,  0.00117298],
        [ 0.01404691,  0.00380778,  0.04436813, -0.01782879,  0.00354445],
        [ 0.01532967,  0.00774736,  0.04863838, -0.02189689,  0.00621756]],

       [[ 0.0035861 , -0.00118294,  0.01193712, -0.0033598 , -0.00072896],
        [ 0.00803438, -0.00130156,  0.02586317, -0.00806076, -0.00040069],
        [ 

In [12]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    # lentgh 1 for batch 1, lentgh 2 for batch 2
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(3, 5, 5), dtype=float32)
array([[[ -3.21578860e-01,   1.75165832e-01,   7.27415234e-02,
          -2.80022789e-02,  -8.90052970e-03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -1.30618246e-05,   8.38171504e-03,   6.79395441e-03,
           5.89703859e-06,   7.36367980e-11],
        [ -1.45217164e-05,   4.90243221e-03,   3.75976134e-03,
           8.18422052e-07,   1.72205512e-12],
        [ -1.46848579e-05,   2.33063614e-03,   2.08668527e-03,
           1.07676804e-07,   3.27396699e-14],
        [  0.00000000e+00,   0.00000000e+0

In [13]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[  1.38200223e-01,   1.50990248e-01,  -5.75374588e-02,
           6.82040527e-02,  -5.39839566e-02],
        [  1.36694703e-02,   1.78428009e-01,  -6.84467959e-04,
           8.35042968e-02,  -5.65271229e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  1.07994011e-13,   8.57330742e-04,  -1.74849501e-12,
           6.04281881e-16,  -5.07376674e-15],
        [  6.20634805e-15,   2.01033225e-04,  -1.86323023e-14,
           6.05210917e-16,  -5.16752524e-15],
        [  3.41355337e-16,   4.68419385e-05,  -1.50983279e-16,
           6.05226217e-16,  -5.23627933e-15],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.

*Seq2Seq Exercise

In [14]:
#flattern based softmax
hidden_size = 3
squence_length = 5
batch_size = 3
num_classes = 5

pp.pprint(x_data)
x_data = x_data.reshape(-1,hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype = np.float32).reshape(hidden_size,
                                                     num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes)
# batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [ 15.,  16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.,  24.]],

       [[ 25.,  26.,  27.,  28.,  29.],
        [ 30.,  31.,  32.,  33.,  34.],
        [ 35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.],
        [ 45.,  46.,  47.,  48.,  49.]],

       [[ 50.,  51.,  52.,  53.,  54.],
        [ 55.,  56.,  57.,  58.,  59.],
        [ 60.,  61.,  62.,  63.,  64.],
        [ 65.,  66.,  67.,  68.,  69.],
        [ 70.,  71.,  72.,  73.,  74.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.],
       [ 24.,  25.,  26.],
       [ 27.,  28.,  29.],
       [ 30.,  31.,  32.],
       [ 33.,  34.,  35.],
       [ 36.,  37.,  38.],
       [ 39.,  40.,  41.],
 

In [15]:
# [batch_size, sequence_length]
y_data = tf.constant([[1,1,1]])

# [batch_size, sequence_length, emb_dim ]
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1,1,1]], dtype =tf.float32)
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = prediction, 
                                                targets = y_data, weights = weights )
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.596759


In [16]:
# [batch_size, sequence_length]
y_data = tf.constant([[1,1,1]])

# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0.3,0.7],[0.3,0.7],[0.3,0.7]]], dtype = tf.float32)
prediction2 = tf.constant([[[0.1,0.9],[0.1,0.9],[0.1,0.9]]], dtype = tf.float32)
prediction3 = tf.constant([[[1,0],[1,0],[1,0]]], dtype = tf.float32)
prediction4 = tf.constant([[[0,1],[0,1],[0,1]]], dtype = tf.float32)

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
     "Loss2: ", sequence_loss2.eval(),
     "Loss3: ", sequence_loss3.eval(),
     "Loss4: ", sequence_loss4.eval())

Loss1:  0.513015 Loss2:  0.371101 Loss3:  1.31326 Loss4:  0.313262
